In [1]:
import pandas as pd

bronze_path = "/lakehouse/default/Files/bronze/openaq/nyc_2025/measurements.parquet"

df_bronze = pd.read_parquet(bronze_path)

df_bronze.info()


StatementMeta(, 3a147b51-cab8-463b-9b21-34aac740c5c9, 3, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   datetime   4000 non-null   datetime64[ns, UTC]
 1   parameter  4000 non-null   object             
 2   value      4000 non-null   float64            
 3   unit       4000 non-null   object             
 4   sensor_id  4000 non-null   int64              
 5   location   4000 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(3)
memory usage: 187.6+ KB


In [4]:
df_silver = df_bronze.copy()

# datetime → datetime64
df_silver["datetime"] = pd.to_datetime(df_silver["datetime"], utc=True)

# value → float
df_silver["value"] = pd.to_numeric(df_silver["value"], errors="coerce")

# базовая очистка
df_silver = df_silver.dropna(subset=["value"])


StatementMeta(, 3a147b51-cab8-463b-9b21-34aac740c5c9, 6, Finished, Available, Finished)

In [7]:
import numpy as np

conditions = [
    df_silver['location'].str.contains("Bayside", na=False),
    df_silver['location'].str.contains("Weehawken", na=False)
]
choices = [30, 265]

df_silver['taxi_zone_id'] = np.select(conditions, choices, default=None)

df_with_zones_spark = spark.createDataFrame(df_silver)
df_with_zones_spark.write.format("delta").mode("overwrite").saveAsTable("silver_air_with_taxi_zones")

display(df_silver.head())

StatementMeta(, 3a147b51-cab8-463b-9b21-34aac740c5c9, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e4ee91aa-5473-442c-9947-8d37dca70f64)

In [8]:
import numpy as np

conditions = [
    df_silver['location'].str.contains("Bayside", na=False),
    df_silver['location'].str.contains("Weehawken", na=False)
]
choices = [30, 265]

df_silver['taxi_zone_id'] = np.select(conditions, choices, default=np.nan)

df_with_zones_spark = spark.createDataFrame(df_silver)

target_path = "Files/silver/openaq/nyc_2025/silver_air_with_taxi"

df_with_zones_spark.write.mode("overwrite").parquet(target_path)

print(f"Файлы успешно сохранены в: {target_path}")
display(df_silver.head())

StatementMeta(, 3a147b51-cab8-463b-9b21-34aac740c5c9, 10, Finished, Available, Finished)

Файлы успешно сохранены в: Files/silver/openaq/nyc_2025/silver_air_with_taxi


SynapseWidget(Synapse.DataFrame, 05a42f7f-05b5-4e4e-b7a9-f7a989297bde)